# Data Engineering

In [75]:
import pandas as pd
import numpy as np

import fundamentus

from google.cloud import storage
import os
import requests
import requests_cache
import logging
import time
from datetime import date

In [3]:
credentials_path = 'datascience-capstone-project-05b1642f45c3.json'

## 1 - Data Collection

For the data gathering, two repositories on Github were combined, so that the historical data could be scraped from the web. This Raw data is beeing stored in the cloud (GCP) and are the main data used for the project.

Repositories utilized:
* https://github.com/mv/fundamentus-api
* https://github.com/Victorcorcos/bovespa-winner

In [178]:
def perc_to_float(val):
    """
    Percent to float
      - replace string in pt-br to float
      - from '45,56%' to 0.4556
    Input:
        (DataFrame, column_name)
    """

    res = val
    res = res.replace( to_replace=r'[%]', value='' , regex=True )
    res = res.replace( to_replace=r'[.]', value='' , regex=True )
    res = res.replace( to_replace=r'[,]', value='.', regex=True )
    res = res.astype(float) / 100

    return res

def _rename_cols(data):
    """
    Rename columns in DataFrame
      - use a valid Python identifier
      - so each column can be a DataFrame property
      - Example:
          df.pl > 0
    """

    df2 = pd.DataFrame()

    ## Fix: rename columns
    df2['cotacao'  ] = data['Cotação'          ]
    df2['pl'       ] = data['P/L'              ]
    df2['pvp'      ] = data['P/VP'             ]
    df2['psr'      ] = data['PSR'              ]
    df2['dy'       ] = data['Div.Yield'        ]
    df2['pa'       ] = data['P/Ativo'          ]
    df2['pcg'      ] = data['P/Cap.Giro'       ]
    df2['pebit'    ] = data['P/EBIT'           ]
    df2['pacl'     ] = data['P/Ativ Circ.Liq'  ]
    df2['evebit'   ] = data['EV/EBIT'          ]
    # df2['evebitda' ] = data['EV/EBITDA'        ]
    df2['mrgebit'  ] = data['Mrg Ebit'         ]
    df2['mrgliq'   ] = data['Mrg. Líq.'        ]
    df2['roic'     ] = data['ROIC'             ]
    df2['roe'      ] = data['ROE'              ]
    df2['liqc'     ] = data['Liq. Corr.'       ]
    df2['liq2m'    ] = data['Liq.2meses'       ]
    df2['patrliq'  ] = data['Patrim. Líq'      ]
    df2['divbpatr' ] = data['Dív.Brut/ Patrim.']
    df2['c5y'      ] = data['Cresc. Rec.5a'    ]

    return df2

def get_resultado_raw(url):
    """
    Get data from fundamentus:
      URL:
        http://fundamentus.com.br/resultado.php
    RAW:
      DataFrame preserves original HTML header names
    Output:
      DataFrame
    """

    ##
    ## Busca avançada por empresa
    ##
    # url = 'http://www.fundamentus.com.br/resultado.php'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01',
           'Accept-Encoding': 'gzip, deflate',
           }

    with requests_cache.enabled():
        content = requests.get(url, headers=hdr)

        if content.from_cache:
            logging.debug('.../resultado.php: [CACHED]')
        else: # pragma: no cover
            logging.debug('.../resultado.php: sleeping...')
            time.sleep(.500) # 500 ms


    ## parse + load
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]

    ## Fix: percent string
    df['Div.Yield']     = perc_to_float( df['Div.Yield']     )
    df['Mrg Ebit']      = perc_to_float( df['Mrg Ebit']      )
    df['Mrg. Líq.']     = perc_to_float( df['Mrg. Líq.']     )
    df['ROIC']          = perc_to_float( df['ROIC']          )
    df['ROE']           = perc_to_float( df['ROE']           )
    df['Cresc. Rec.5a'] = perc_to_float( df['Cresc. Rec.5a'] )

    ## index by 'Papel', instead of 'int'
    df.index = df['Papel']
    df.drop('Papel', axis='columns', inplace=True)
    df.sort_index(inplace=True)

    ## naming
    df.name = 'Fundamentus: HTML names'
    df.columns.name = 'Multiples'
    df.index.name = 'papel'

    ## return sorted by 'papel'
    return df


def get_resultado(url):
    """
    Data from fundamentus, fixing header names.
      URL:
        given from the user
      Obs:
        DataFrame uses short header names
    Output:
      DataFrame
    """

    ## get RAW data
    data1 = get_resultado_raw(url)

    ## rename!
    data2 = _rename_cols(data1)

    ## metadata
    data2.name = 'Fundamentus: short names'
    data2.columns.name = 'Multiples'
    data2.index.name = 'papel'

    ## remove duplicates
#   df = data2.drop_duplicates(subset=['cotacao','pl','pvp'], keep='last')
    df = data2.drop_duplicates(keep='first')

    return df


In [5]:
def initialize_bucket(credentials_path, create_bucket=False):

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

    client = storage.Client()
    bucket = client.bucket('storage-barsianize')
    
    if create_bucket:
        bucket.location = 'US-EAST1'
        bucket.create()

    return client, bucket
    

In [134]:
def get_windows(max, min, window=5, step=1):

    maxes = list(range(min+window,max+1, step))
    mins = list(range(min, max+1-window, step))
    sequences = []
    if len(maxes)==len(mins):
        for i in range(len(maxes)):
            sequence = list(range(mins[i],maxes[i]+1,step))
            sequences.append(sequence)
        return sequences
    else:
        print(f'error!!! min {maxes}, max {maxes}')

### Get historical data

In [180]:
urls = {
    2008: 'https://web.archive.org/web/20080613050801/http://www.fundamentus.com.br/resultado.php',
    2009: 'https://web.archive.org/web/20090123022224/http://www.fundamentus.com.br/resultado.php',
    2010: 'https://web.archive.org/web/20100115191626/http://www.fundamentus.com.br/resultado.php',
    2011: 'https://web.archive.org/web/20110113192117/http://www.fundamentus.com.br/resultado.php',
    2012: 'https://web.archive.org/web/20120106023830/http://www.fundamentus.com.br/resultado.php',
    2013: 'https://web.archive.org/web/20130105004012/http://www.fundamentus.com.br/resultado.php',
    2014: 'https://web.archive.org/web/20140108164618/http://www.fundamentus.com.br/resultado.php',
    2015: 'https://web.archive.org/web/20150119231047/http://www.fundamentus.com.br/resultado.php',
    2016: 'https://web.archive.org/web/20160106101916/http://www.fundamentus.com.br/resultado.php',
    2017: 'https://web.archive.org/web/20170505164235/http://www.fundamentus.com.br/resultado.php',
    2018: 'https://web.archive.org/web/20180105120409/http://www.fundamentus.com.br/resultado.php',
    2019: 'https://web.archive.org/web/20190102202956/http://www.fundamentus.com.br/resultado.php',
    2020: 'https://web.archive.org/web/20200122200313/http://www.fundamentus.com.br/resultado.php',
    2021: 'https://web.archive.org/web/20210227034423/http://www.fundamentus.com.br/resultado.php',
    2022: 'https://web.archive.org/web/20220314021607/http://www.fundamentus.com.br/resultado.php',
    2023: 'http://fundamentus.com.br/resultado.php'
  }
  
years = list(range(2008,2024))

df_full = pd.DataFrame()
for year in years:
    df = get_resultado(urls[year])
    df['year'] = year
    df_full = pd.concat([df_full,df])
    
df_full = df_full.reset_index()

### Get detailed information about each ticker

In [181]:
papeis = df_full['papel'].unique()

df_papeis = pd.DataFrame()
for papel in papeis:
    try:
        df = fundamentus.get_papel(papel)
        df_papeis = pd.concat([df_papeis,df])
    except:
        print(f'fail papel {papel}')

2023-01-04 22:02:00,062 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,126 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,190 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,210 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,275 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,344 [detalhes.get_papel] INFO: detalhes: call: get..._papel()


fail papel ABNB3


2023-01-04 22:02:00,410 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,477 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,545 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,613 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,680 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,760 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,828 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,847 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,912 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:00,981 [detalhes.get_papel] INFO: detalhes: call: get..._papel()


fail papel AGEN11


2023-01-04 22:02:01,047 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,109 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,173 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,237 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,301 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,364 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,440 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,508 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,569 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,634 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,696 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02:01,760 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:02

...

2023-01-04 22:04:29,975 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:30,824 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:31,685 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:32,696 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:33,573 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:34,434 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:35,214 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:36,051 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:36,900 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:37,951 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:38,774 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:39,620 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04

fail papel OGSA3


2023-01-04 22:04:41,390 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:42,269 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:43,131 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:43,981 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:44,812 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:45,679 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:46,562 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:47,416 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:48,277 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:49,294 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:50,307 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04:51,169 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:04

fail papel LIQO3


2023-01-04 22:05:26,708 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:27,855 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:28,696 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:29,536 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:30,371 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:31,239 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:32,259 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:33,115 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:33,895 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:34,954 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:35,773 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05:36,619 [detalhes.get_papel] INFO: detalhes: call: get..._papel()
2023-01-04 22:05

In [182]:
df_full[df_full['papel']=='ITUB4']

Multiples,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,...,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year
1885,ITUB4,37.91,19.86,3.55,0.0,0.0188,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1786,0.0,1.686560e+08,4.886160e+10,0.0,0.1767,2010
2654,ITUB4,40.25,14.55,3.22,0.0,0.0246,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.2210,0.0,2.156390e+08,5.722510e+10,0.0,0.2734,2011
3440,ITUB4,34.66,13.27,2.09,0.0,0.0292,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1572,0.0,2.463310e+08,7.591600e+10,0.0,-0.4082,2012
4236,ITUB4,35.51,14.42,1.93,0.0,0.0325,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1337,0.0,2.728300e+08,8.416040e+10,0.0,-0.4905,2013
5051,ITUB4,31.27,14.83,1.80,0.0,0.0316,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1214,0.0,3.127270e+08,8.733480e+10,0.0,-0.4519,2014
5870,ITUB4,33.10,12.07,1.88,0.0,0.0299,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1559,0.0,4.247690e+08,9.726990e+10,0.0,-0.1391,2015
6689,ITUB4,25.34,6.96,1.43,0.0,0.0584,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.2050,0.0,4.617880e+08,1.080200e+11,0.0,0.2599,2016
7524,ITUB4,38.10,12.31,2.13,0.0,0.0468,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1733,0.0,4.294830e+08,1.175570e+11,0.0,0.1019,2017
8372,ITUB4,45.10,13.70,2.35,0.0,0.0334,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1717,0.0,4.841310e+08,1.255750e+11,0.0,0.0391,2018
9224,ITUB4,35.50,17.59,2.77,0.0,0.0595,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1576,0.0,7.078580e+08,1.255340e+11,0.0,-0.1029,2019


In [183]:
df_papeis.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_med_2m,...,Rec_Servicos_3m,Lucro_Liquido_3m,Disponibilidades,Ativo_Circulante,Div_Bruta,Div_Liquida,Receita_Liquida_12m,EBIT_12m,Receita_Liquida_3m,EBIT_3m
ABCB3,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,0.00,1899-12-30,0.00,0.00,0,...,118020000,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABCB4,ABCB4,PN N2,ABC Brasil PN N2,Intermediários Financeiros,Bancos,17.91,2023-01-04,14.06,22.30,13880200,...,118020000,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABYA3,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,4.91,2010-02-11,0.00,0.00,0,...,NaN,-766000,29556000,498222000,383159000,353603000,250657000,25802000,67990000,18057000
ACES3,ACES3,ON,ARCELORMITTAL INOX BRASIL ON,Siderurgia e Metalurgia,Siderurgia,95.27,2008-04-18,0.00,0.00,0,...,NaN,163789000,922471000,2157470000,279166000,-643305000,4182420000,983634000,949114000,174503000
ACES4,ACES4,PN,ARCELORMITTAL INOX BRASIL PN,Siderurgia e Metalurgia,Siderurgia,94.73,2008-04-24,0.00,0.00,0,...,NaN,163789000,922471000,2157470000,279166000,-643305000,4182420000,983634000,949114000,174503000


In [184]:
df_full.head()

Multiples,papel,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,...,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year
0,ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.0000,0.00,0.0,1.107010e+09,0.00,0.0000,2008
1,ABCB4,9.00,0.00,1.10,0.000,0.0489,0.000,0.00,0.00,0.00,...,0.0000,0.0000,0.0000,0.0000,0.00,4160460.0,1.107010e+09,0.00,0.0000,2008
2,ABNB3,17.35,12.39,3.27,1.845,0.0254,2.590,7.26,8.45,5.04,...,0.2182,0.1490,0.3465,0.2639,3.28,2586090.0,2.733490e+08,0.00,0.1446,2008
3,ABYA3,16.00,25.67,4.13,4.399,0.0079,0.570,2.10,17.45,2.51,...,0.2520,0.1714,0.0356,0.1608,1.94,5915230.0,1.971610e+08,2.40,0.0000,2008
4,ACES3,95.27,8.79,2.28,1.693,0.0000,1.438,6.40,7.20,3.77,...,0.2352,0.1927,0.2690,0.2595,2.05,181013.0,3.105800e+09,0.09,0.1334,2008


### Storing the raw data into Google Cloud

In [186]:
client, bucket = initialize_bucket(credentials_path)

# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('raw/df_full.csv')
blob.upload_from_string(df_full.to_csv(), 'text/csv')
# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('raw/df_papeis.csv')
blob.upload_from_string(df_papeis.to_csv(), 'text/csv')
# upload raw fundamentalist data to Google Cloud Storage
# blob = bucket.blob('raw/categories.csv')
# blob.upload_from_string(categories.to_csv(), 'text/csv')

## 2 - Data Cleansing

### Load data

In [29]:
client, bucket = initialize_bucket(credentials_path)

path = "gs://storage-barsianize/raw/df_full.csv"
df_full =  pd.read_csv(path, index_col=0)

path = "gs://storage-barsianize/raw/df_papeis.csv"
df_papeis = pd.read_csv(path, index_col=0)

Desired information about each ticker

In [30]:
df_papeis.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Cotacao,Data_ult_cot,Min_52_sem,Max_52_sem,Vol_med_2m,...,Rec_Servicos_3m,Lucro_Liquido_3m,Disponibilidades,Ativo_Circulante,Div_Bruta,Div_Liquida,Receita_Liquida_12m,EBIT_12m,Receita_Liquida_3m,EBIT_3m
ABCB3,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,0.00,1899-12-30,0.00,0.0,0,...,118020000.0,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABCB4,ABCB4,PN N2,ABC Brasil PN N2,Intermediários Financeiros,Bancos,17.91,2023-01-04,14.06,22.3,13880200,...,118020000.0,218148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABYA3,ABYA3,ON NM,ABYARA ON NM,Construção Civil,Incorporações,4.91,2010-02-11,0.00,0.0,0,...,NaN,-766000,29556000.0,4.982220e+08,383159000.0,353603000.0,2.506570e+08,25802000.0,67990000.0,18057000.0
ACES3,ACES3,ON,ARCELORMITTAL INOX BRASIL ON,Siderurgia e Metalurgia,Siderurgia,95.27,2008-04-18,0.00,0.0,0,...,NaN,163789000,922471000.0,2.157470e+09,279166000.0,-643305000.0,4.182420e+09,983634000.0,949114000.0,174503000.0
ACES4,ACES4,PN,ARCELORMITTAL INOX BRASIL PN,Siderurgia e Metalurgia,Siderurgia,94.73,2008-04-24,0.00,0.0,0,...,NaN,163789000,922471000.0,2.157470e+09,279166000.0,-643305000.0,4.182420e+09,983634000.0,949114000.0,174503000.0


In [31]:
info_papeis = ['Papel','Tipo', 'Empresa', 'Setor', 'Subsetor','Data_ult_cot']
df_papeis_clean = df_papeis[info_papeis]

df_papeis_clean = df_papeis_clean.dropna()
df_papeis_clean = df_papeis_clean.drop_duplicates()

In [43]:
df_papeis_clean['Data_ult_cot'] = pd.to_datetime(df_papeis_clean['Data_ult_cot'])

In [44]:
df_completed = df_papeis_clean.merge(df_full, how='left', left_index=True, right_on='papel').drop_duplicates()

In [45]:
df_full.shape

(13339, 21)

In [46]:
df_completed.isna().mean().sort_values(ascending=False)

Papel           0.0
pebit           0.0
c5y             0.0
divbpatr        0.0
patrliq         0.0
liq2m           0.0
liqc            0.0
roe             0.0
roic            0.0
mrgliq          0.0
mrgebit         0.0
evebit          0.0
pacl            0.0
pcg             0.0
Tipo            0.0
pa              0.0
dy              0.0
psr             0.0
pvp             0.0
pl              0.0
cotacao         0.0
papel           0.0
Data_ult_cot    0.0
Subsetor        0.0
Setor           0.0
Empresa         0.0
year            0.0
dtype: float64

In [47]:
client, bucket = initialize_bucket(credentials_path)

# upload raw fundamentalist data to Google Cloud Storage
blob = bucket.blob('trusted/df_completed.csv')
blob.upload_from_string(df_completed.to_csv(), 'text/csv',)

## 3 - Data Preparation

In [48]:
client, bucket = initialize_bucket(credentials_path)

In [68]:
path = "gs://storage-barsianize/trusted/df_completed.csv"
df_completed =  pd.read_csv(path, index_col=0)
df_completed['Data_ult_cot'] = pd.to_datetime(df_completed['Data_ult_cot'])

In [69]:
df_completed.head()

,Papel,Tipo,Empresa,Setor,Subsetor,Data_ult_cot,papel,cotacao,pl,pvp,...,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year
0,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,1.107010e+09,0.0,0.0000,2008
736,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1553,0.0,0.0,1.158760e+09,0.0,0.0000,2009
1478,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1016,0.0,0.0,1.182180e+09,0.0,-0.1233,2010
2237,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1521,0.0,0.0,1.312080e+09,0.0,0.1043,2011
3020,ABCB3,ON N2,ABC Brasil ON N2,Intermediários Financeiros,Bancos,1899-12-30,ABCB3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1571,0.0,0.0,1.460150e+09,0.0,0.1272,2012


In [70]:
df_completed[df_completed['year']==2023]['Papel'].unique()

array(['ABCB3', 'ABCB4', 'ABYA3', 'ACES3', 'ACES4', 'AEDU11', 'AELP3',
       'AESL3', 'AFLU3', 'AFLU5', 'AGIN3', 'AGRO3', 'AHEB3', 'AHEB5',
       'AHEB6', 'ALLL11', 'ALLL3', 'ALLL4', 'ALPA3', 'ALPA4', 'AMIL3',
       'AMPI3', 'APTI4', 'ARCZ3', 'ARCZ6', 'AVIL3', 'AZEV3', 'AZEV4',
       'BAHI11', 'BAHI3', 'BAHI4', 'BAHI5', 'BALM3', 'BALM4', 'BAUH4',
       'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BDLL3', 'BDLL4', 'BEEF3',
       'BEES3', 'BEMA3', 'BGIP3', 'BGIP4', 'BICB3', 'BICB4', 'BIOM3',
       'BIOM4', 'BISA3', 'BMEB3', 'BMEB4', 'BMEF3', 'BMIN3', 'BMIN4',
       'BMKS3', 'BNBR3', 'BNBR4', 'BNCA3', 'BOBR3', 'BOBR4', 'BOVH3',
       'BPAR3', 'BPIA3', 'BPNM3', 'BRAP3', 'BRAP4', 'BRGE11', 'BRGE12',
       'BRGE3', 'BRGE5', 'BRGE6', 'BRGE7', 'BRGE8', 'BRIV3', 'BRIV4',
       'BRKM3', 'BRKM5', 'BRKM6', 'BRML3', 'BRSR3', 'BRSR4', 'BRSR5',
       'BRSR6', 'BSCT3', 'BSCT5', 'BSCT6', 'BTTL3', 'BUET3', 'BUET4',
       'CALI3', 'CALI4', 'CAMB3', 'CAMB4', 'CASN3', 'CBEE3', 'CCHI3',
       'CCIM3',

In [73]:
df_completed['Data_ult_cot_year'] = df_completed['Data_ult_cot'].dt.year

In [82]:
df_actual = df_completed[df_completed['Data_ult_cot_year'] == pd.to_datetime(date.today()).year]

In [87]:
tickers = df_actual['Papel'].unique()

In [88]:
tickers

array(['ABCB4', 'AGRO3', 'ALPA3', 'ALPA4', 'AZEV3', 'AZEV4', 'BAHI3',
       'BALM3', 'BALM4', 'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BDLL4',
       'BEEF3', 'BEES3', 'BGIP4', 'BIOM3', 'BMEB3', 'BMEB4', 'BMIN3',
       'BMIN4', 'BMKS3', 'BOBR4', 'BRAP3', 'BRAP4', 'BRGE11', 'BRGE3',
       'BRIV3', 'BRIV4', 'BRKM3', 'BRKM5', 'BRML3', 'BRSR3', 'BRSR5',
       'BRSR6', 'CALI3', 'CAMB3', 'CBEE3', 'CCRO3', 'CEBR3', 'CEBR5',
       'CEBR6', 'CEDO4', 'CEEB3', 'CGAS3', 'CGAS5', 'CGRA3', 'CGRA4',
       'CLSC3', 'CMIG3', 'CMIG4', 'COCE5', 'CPFE3', 'CPLE3', 'CPLE6',
       'CRIV3', 'CRIV4', 'CSAB3', 'CSAB4', 'CSAN3', 'CSMG3', 'CSNA3',
       'CSRN3', 'CSRN5', 'CSRN6', 'CTKA4', 'CTNM4', 'CTSA3', 'CTSA4',
       'CYRE3', 'DASA3', 'DOHL4', 'EALT3', 'EALT4', 'EKTR3', 'EKTR4',
       'ELET3', 'ELET6', 'EMAE4', 'EMBR3', 'ENBR3', 'ENGI3', 'ENGI4',
       'EQTL3', 'ESTR4', 'ETER3', 'EUCA3', 'EUCA4', 'EVEN3', 'EZTC3',
       'FESA3', 'FESA4', 'FHER3', 'GEPA3', 'GEPA4', 'GFSA3', 'GGBR3',
       'GGBR4', 'GO

In [92]:
df_actual.columns

Index(['Papel', 'Tipo', 'Empresa', 'Setor', 'Subsetor', 'Data_ult_cot',
       'papel', 'cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg', 'pebit',
       'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc', 'liq2m',
       'patrliq', 'divbpatr', 'c5y', 'year', 'Data_ult_cot_year'],
      dtype='object')

In [93]:
indicators = ['cotacao', 'pl', 'pvp', 'psr', 'dy', 'pa', 'pcg', 'pebit',
              'pacl', 'evebit', 'mrgebit', 'mrgliq', 'roic', 'roe', 'liqc', 'liq2m',
              'patrliq', 'divbpatr', 'c5y', 'year']

In [94]:
df_indicators = df_actual[indicators]

In [103]:
data = df_indicators[df_actual['Papel']==tickers[2]].sort_values('year', ascending=False)

max = data['year'].max()
min = data['year'].min()
windows = get_windows(max, min, window=5, step=1)

In [125]:
data.head()

,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y,year
12400,11.23,17.58,1.31,1.933,0.0000,0.970,5.66,12.59,7.01,12.82,0.1536,0.1090,0.0864,0.0745,1.76,106685.0,5.860170e+09,0.07,0.0048,2023
11435,19.20,19.02,3.86,3.321,0.0175,1.633,-12.29,19.35,-9.79,18.65,0.1716,0.1754,0.0996,0.2031,0.75,121021.0,3.395640e+09,0.03,-0.0031,2022
10519,30.25,126.89,6.03,5.288,0.0000,3.695,12.68,32.07,20.14,31.26,0.1649,0.0377,0.1517,0.0475,2.10,200001.0,2.948730e+09,0.08,-0.0369,2021
9653,30.01,76.96,6.96,4.251,0.0000,4.046,13.41,36.76,26.17,36.53,0.1156,0.0498,0.1400,0.0905,2.18,591463.0,2.535080e+09,0.17,-0.0054,2020
8799,16.36,25.26,3.42,2.011,0.0262,1.978,6.50,17.85,10.42,17.78,0.1127,0.0773,0.1536,0.1354,2.05,34826.9,2.249190e+09,0.27,-0.0011,2019


In [184]:
def get_variations(data, windows):
    
    data_line_full = pd.DataFrame()
    cols = ['cotacao','pl','dy','roe','patrliq','divbpatr','pvp']
    
    for window in windows:
        window.sort(reverse=True)

        data_window = data[np.isin(data['year'],window)]
        data_window = data_window.sort_values(by='year', ascending=False)

        data_line = {}

        

        for col in cols:
            data_line[col] = data_window[col].iloc[0]
            data_line[f'{col}_rel'] = data_window[col].iloc[0] / data_window[col].iloc[-1]
            data_line[f'{col}_median'] = data_window[col].median()
            data_line[f'{col}_median_rel'] = data_window[col].median()/ data_window[col].iloc[-1]
            data_line[f'{col}_mean'] = data_window[col].mean()
            data_line[f'{col}_mean_rel'] = data_window[col].mean()/ data_window[col].iloc[-1]
            data_line[f'{col}_std'] = data_window[col].std()
            data_line[f'{col}_std_rel'] = data_window[col].std()/ data_window[col].iloc[-1]

            data_line[f'{col}_spread'] = (data_window[col].max() - data_window[col].min()) / data_window[col].mean()

        data_line_full = pd.concat([data_line_full, pd.DataFrame(data_line, index=[0])])


    return data_line_full
        


In [185]:
get_variations(data, windows)

,cotacao,cotacao_rel,cotacao_median,cotacao_median_rel,cotacao_mean,cotacao_mean_rel,cotacao_std,cotacao_std_rel,cotacao_spread,pl,...,divbpatr_spread,pvp,pvp_rel,pvp_median,pvp_median_rel,pvp_mean,pvp_mean_rel,pvp_std,pvp_std_rel,pvp_spread
0,11.59,0.079937,48.50,0.334506,67.466,0.465315,62.285502,0.429585,1.982332,12.99,...,0.578512,2.84,0.969283,2.84,0.969283,2.426,0.827986,0.890382,0.303885,0.915087
0,14.95,0.308247,14.95,0.308247,41.458,0.854804,47.126994,0.971691,2.647257,21.33,...,0.588235,3.60,3.600000,2.84,2.840000,2.560,2.560000,1.025378,1.025378,1.015625
0,12.32,0.101818,12.32,0.101818,34.222,0.282826,48.532075,0.401092,3.207001,17.35,...,0.555556,2.97,1.387850,2.97,1.387850,2.954,1.380374,0.539426,0.252068,0.494245
0,7.05,0.626667,11.59,1.030222,11.432,1.016178,2.847142,0.253079,0.691043,12.38,...,0.555556,1.74,0.540373,2.97,0.922360,2.874,0.892547,0.696979,0.216453,0.647182
0,9.56,0.824849,11.59,1.000000,11.094,0.957204,2.971739,0.256405,0.712097,14.54,...,0.423729,2.34,0.823944,2.84,1.000000,2.698,0.950000,0.698871,0.246081,0.689400
0,12.86,0.860201,12.32,0.824080,11.348,0.759064,3.077137,0.205829,0.696158,16.73,...,0.436508,3.06,0.850000,2.97,0.825000,2.742,0.761667,0.716743,0.199095,0.678337
0,17.09,1.387175,12.32,1.000000,11.776,0.955844,3.773437,0.306286,0.852582,19.36,...,0.338346,3.69,1.242424,2.97,1.000000,2.760,0.929293,0.744278,0.250599,0.706522
0,16.36,2.320567,12.86,1.824113,12.584,1.784965,4.313007,0.611774,0.797839,25.26,...,0.343511,3.42,1.965517,3.06,1.758621,2.850,1.637931,0.801062,0.460380,0.684211
0,30.01,3.139121,16.36,1.711297,17.176,1.796653,7.778434,0.813644,1.190615,76.96,...,0.511811,6.96,2.974359,3.42,1.461538,3.894,1.664103,1.787255,0.763784,1.186441
0,30.25,2.352255,17.09,1.328927,21.314,1.657387,8.205591,0.638071,0.815896,126.89,...,1.018519,6.03,1.970588,3.69,1.205882,4.632,1.513725,1.746531,0.570762,0.841969
